In [1]:
# -*- coding: utf-8 -*-
#每页50个文件
import requests
from lxml import etree
import csv
import time
import random
from collections import OrderedDict
import codecs
from datetime import date
from multiprocessing.dummy import Pool as Threadpool
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.expected_conditions import alert_is_present
from selenium.webdriver.common.alert import Alert
import ast
import json
import re
import os
import string
import cookielib
import urllib2

# 判断是否爬取页面信息，为1则重新爬并缓存页面，为0则直接从缓存页面捕获信息。
is_crwal = 0

class Wenkuspider(object):
    def __init__(self):
        self.__file__ = r'C:/Users/wz/Desktop/lvmeng/paloalt_submit00.json'
        self.__link__ = r"https://openapi.book118.com/login/login/index.html"
        self.__loadeddir__ = 'loadedhtml0'
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}
        # 如果从某处断线了，可以更改起始的url地址
        #self.start_url = "http://www.cnvd.org.cn/flaw/list.htm"
        self.count = 0
        self.cookies = self.get_cookies()


    # 获取网站cookie
    def get_cookies(self):
        # 构建一个CookieJar对象实例来保存cookie
        cookiejar = cookielib.CookieJar()
        # 使用HTTPCookieProcessor()来创建cookie处理器对象，参数为CookieJar()对象
        handler=urllib2.HTTPCookieProcessor(cookiejar)
        # 通过 build_opener() 来构建opener
        opener = urllib2.build_opener(handler)
        # 4. 以get方法访问页面，访问之后会自动保存cookie到cookiejar中
        opener.open(self.__link__)
        # 可以按标准格式将保存的Cookie打印出来
        cookieStr = ''
        for item in cookiejar:
            cookieStr += "'"+item.name + "':'" + item.value + "',"
        cookie = ast.literal_eval('{'+cookieStr+'}')
#         print(cookie)
        return cookie
 
    def run(self):        
        chrome_options = Options()
        # 谷歌文档提到需要加上这个属性来规避bug
        chrome_options.add_argument('--disable-gpu') 
        # 加上下面两行，解决报错
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
#         chrome_options.add_argument("--headless")
        chrome_options.binary_location =  r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
        driver = webdriver.Chrome(chrome_options=chrome_options)
        # 设置每次ChromeDriver访问的初始页面用来更新cookie，以绕过cnvd爬虫限制 OK
        driver.get("https://openapi.book118.com/login/login/index.html")
        time.sleep(3)
#         driver.set_window_size(1920, 1080)
        raw_input()
#         # 进入“我的文档”
#         driver.find_element_by_xpath("//ul[@id='nav']/li[4]/a/span").click()
#         time.sleep(3)
#         # 进入“系统失败可发布”
#         driver.find_element_by_xpath('//*[@id="broadside"]/div[1]/ul/li[3]/ul/li[5]/a/em').click()
        time.sleep(3)
#         # 调整网页尺寸
#         driver.execute_script("document.body.style.zoom='0.5'") #缩小
#         time.sleep(3)
        # 切换到frame框架
        driver.switch_to.frame('doc_index_iframe')
#         time.sleep(3)        
        num = 2
#         num0 = 1
        time.sleep(3)
#         print(num)
        # 1-20页
        for i in range(15,27):
            num0 = 1
            # 动态页面中的内容
            zicontent = driver.page_source
            html = etree.HTML(zicontent)
            # 50个文件名组成的列表
            names = html.xpath('//*[@id="table"]/div[1]/table/tbody/tr/td[3]/a/text()')
            for name in names:
                name = name.strip()
                # 出现弹窗
                xpatha = '#table > div.container > table > tbody > tr:nth-child('
                xpathb = str(num0)
                xpathc = ') > td.col-operate.col-colspan > ul > li:nth-child(1) > a'
                # 点开“修改”按钮
                trial = driver.find_element_by_css_selector(xpatha+xpathb+xpathc)
                driver.execute_script('arguments[0].click()', trial)
                time.sleep(2)
                driver.switch_to_default_content()

                time.sleep(1)
                # 切换到frame框架layui-layer-iframe2
                framename = 'layui-layer-iframe'+str(int(num))
                driver.switch_to.frame(framename)
                driver.find_element_by_name('keywords').click()     #先点击输入文本的输入框
    #             time.sleep(2)
                driver.find_element_by_name('keywords').clear()
                driver.find_element_by_name('keywords').send_keys(name)
#                 time.sleep(0.5)

    #             time.sleep(2)layui-layer-iframe4            
                # 点开“summary”按钮
                trials = driver.find_element_by_css_selector('#summary')
                driver.execute_script('arguments[0].click()', trials)
    #             driver.find_element_by_id('summary').click()     #先点击输入文本的输入框
    #             time.sleep(2)
                driver.find_element_by_id('summary').clear()
                driver.find_element_by_id('summary').send_keys(name+name+name+name)
#                 time.sleep(0.5)            
                trials1010 = driver.find_element_by_css_selector('#edit_btn_sub')
                driver.execute_script('arguments[0].click()', trials1010)
#                 driver.find_element_by_xpath('//*[@id="edit_btn_sub"]').click()
                time.sleep(2) 
                driver.switch_to_default_content()
                # 切换到frame框架
                driver.switch_to.frame('doc_index_iframe')
                time.sleep(0.5)
    #             # 点开“申请审核按钮”按钮
    #             xpatha = '#table > div.container > table > tbody > tr:nth-child('
    #             xpathb = str(num0)
    #             xpathc = ') > td.col-operate.col-colspan > ul > li:nth-child(1) > a'            
    #             trial = driver.find_element_by_css_selector(xpatha+xpathb+xpathc)
    #             driver.execute_script('arguments[0].click()', trial)
    #             time.sleep(2)
    #             driver.switch_to_default_content()
    #             # 切换到frame框架layui-layer-iframe2
    #             num = num+2
    #             framename = 'layui-layer-iframe'+str(int(num))
    #             driver.switch_to.frame(framename) # layui-layer-iframe9
    #             driver.find_element_by_name('summary').click()     #先点击输入文本的输入框
    # #             time.sleep(2)
    #             driver.find_element_by_name('summary').send_keys(name+name+name+name)
    #             time.sleep(0.5)            
    #             driver.find_element_by_xpath('//*[@id="change_audit_btn_sub"]').click()
    #             time.sleep(2) 
    #             driver.switch_to_default_content()
    #             # 切换到frame框架
    #             driver.switch_to.frame('doc_index_iframe')
    #             time.sleep(2)
    #             time.sleep(5)
    #             driver.find_element_by_xpath(xpatha+xpathb+xpathc).click()
                num = num + 2
                num0 = num0+1
                print(num0)
                print(num)                    
            # 点开“下一页”按钮
            trials = driver.find_element_by_css_selector('#paging > div > div.pagelist > a:nth-child(8)')
            driver.execute_script('arguments[0].click()', trials)
            time.sleep(2)
    #         print(aa)
#         print(num0)
        print(num)
        print('自动审核完毕')
#         driver.quit()
#         raw_input()
#       

if __name__ == "__main__":
    a = Wenkuspider()
    pool = Threadpool(1)  # 单线程跑的慢，但是基本很稳定没有被cnvd封杀。本次跑完整个cnvd库大概十天左右，期间网络波动中断需要维护代码，及爬虫重新爬的页面范围。
    a.run()
    # pool.map(a.run(),self.parse())
    pool.close()
    pool.join()

c:\python27\lib\site-packages\ipykernel_launcher.py:69: DeprecationWarning: use options instead of chrome_options


c:\python27\lib\site-packages\ipykernel_launcher.py:109: DeprecationWarning: use driver.switch_to.default_content instead
c:\python27\lib\site-packages\ipykernel_launcher.py:134: DeprecationWarning: use driver.switch_to.default_content instead


2
4
3
6
4
8
5
10
6
12
7
14
8
16
9
18
10
20
11
22
12
24
13
26
14
28
15
30
16
32
17
34
18
36
19
38
20
40
21
42
22
44
23
46
24
48
25
50
26
52
27
54
28
56
29
58
30
60
31
62
32
64
33
66
34
68
35
70
36
72
37
74
38
76
39
78
40
80
41
82
42
84
43
86
44
88
45
90
46
92
47
94
48
96
49
98
50
100
51
102
2
104
3
106
4
108
5
110
6
112
7
114
8
116
9
118
10
120
11
122
12
124
13
126
14
128
15
130
16
132
17
134
18
136
19
138
20
140
21
142
22
144
23
146
24
148
25
150
26
152
27
154
28
156
29
158
30
160
31
162
32
164
33
166
34
168
35
170
36
172
37
174
38
176
39
178
40
180
41
182
42
184
43
186
44
188
45
190
46
192
47
194


NoSuchFrameException: Message: layui-layer-iframe194


In [4]:
for i in range(1,5):
    print(i)

1
2
3
4
